In [ ]:
import google.auth
from google.cloud import bigquery
import pandas as pd
import pandas_gbq

In [ ]:
def create_big_query_client():
    credentials, project_id = google.auth.default()

    return bigquery.Client(
      credentials=credentials,
      project=project_id)

In [ ]:
client = create_big_query_client()

In [ ]:
get_last_checker_result_query = """
SELECT DISTINCT
fullVisitorId,
REGEXP_EXTRACT(
    LAST_VALUE(hits.page.pagePath) OVER (
      PARTITION BY fullVisitorId ORDER BY (visitStartTime + hits.time) ASC
      ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING),
  r"/transition-check/results\?c\[.??\]\=(.*)") AS pagePath,
LAST_VALUE(hits.page.pagePath) OVER (
  PARTITION BY fullVisitorId ORDER BY (visitStartTime + hits.time) ASC
  ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS pagePath_old
FROM
`govuk-bigquery-analytics.87773428.ga_sessions_*` AS sessions
CROSS JOIN
  UNNEST(sessions.hits) AS hits
WHERE
    _TABLE_SUFFIX between '20201215' and '20201215'
    AND hits.page.pagePath LIKE '/transition-check/results?%'
"""

In [ ]:
checker_results_df = pandas_gbq.read_gbq(get_last_checker_result_query)

In [ ]:
checker_results_df.head()

In [ ]:
checker_results_df.to_csv('latest_checker_results_20201215.csv', index=False)

In [ ]:
def split_page_path(page_path):
    try:
        return page_path.split('&c[]=')
    except AttributeError:
        print(page_path)

In [ ]:
checker_results_df[checker_results_df['pagePath'].isnull()]

In [ ]:
checker_results_df['answers'] = checker_results_df['pagePath'].map(split_page_path)

In [ ]:
checker_results_df.head()

In [ ]:
checker_results_df['answers'][0]